In [4]:
!pip install langchain-community==0.2.1 langchain-core==0.2.1
!pip install langchain-huggingface==0.0.2
!pip install pypdf
!pip install unstructured
!pip install sentence_transformers
!pip install faiss-cpu
!pip install tiktoken
#!wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.0/auto_gptq-0.4.0+cu118-cp310-cp310-linux_x86_64.whl
#!pip install -qqq auto_gptq-0.4.0+cu118-cp310-cp310-linux_x86_64.whl --progress-bar off
!pip install -U langchain-community
#!pip install --upgrade langchain sentence-transformers
!pip install accelerate

  Using cached langchain_community-0.2.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.2.3
    Uninstalling langchain-community-0.2.3:
      Successfully uninstalled langchain-community-0.2.3
  Using cached langchain_community-0.2.3-py3-none-any.whl (2.2 MB)
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.2.1
    Uninstalling langchain-community-0.2.1:
      Successfully uninstalled langchain-community-0.2.1


In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.chains.question_answering import load_qa_chain

from langchain.vectorstores import FAISS
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from huggingface_hub import hf_hub_download

import torch
#from auto_gptq import AutoGPTQForCausalLM
from langchain import PromptTemplate
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, TextStreamer, pipeline,AutoModelForCausalLM


DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
import os

In [6]:
#Load the pdf
loader = PyPDFLoader("/content/HLPE-Report-12_AR.pdf")
data = loader.load()
#Split the pdf into chunks
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs=text_splitter.split_documents(data)

In [7]:
docs[4]

Document(page_content='أعضاء فريق الخبراء الرفيع المستوى المعني بالأمن الغذائي والتغذية المسؤولون عن المشروع  \n \nJessica Fanzo (قائدة الفريق) \nMandana Arabi \nBarbara Burlingame  \nLawrence Haddad  \nSimon Kimenju  \nGregory Miller  \nFengying Nie  \nElisabetta Recine  \nLluís Serra -Majem  \nDipa Sinha   \n \nمنسق فريق الخبراء الرفيع المستوى المعني بالأمن الغذائي والتغذية \nNathanaël Pingault  \nاعتمدت اللجنة التوجيهية لفريق الخبراء الرفيع المستوى المعني بالأمن الغذائي والتغذية هذا التقرير الذي أعد ه الفريق.', metadata={'source': '/content/HLPE-Report-12_AR.pdf', 'page': 2})

In [8]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')#'BAAI/bge-base-en-v1.5') #Model's job is to perform embeddings

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
docsearch=FAISS.from_documents(docs,embeddings)

In [10]:
query="what is marketing ?"
answers=docsearch.similarity_search(query)

In [11]:
answers[0]

Document(page_content='Story, M. & French, S.  2004. Food advertising and marketing directed at children and \nadolescents in the US. International Journal of Behavioral Nutrition and Physical Activity . \n1(1): 3. \n . PLoS Med, 2012. Big Food, Food Systems, and Global Health Stuckler, D. & Nestle, M.\n.pmed.1001242 https://doi.org/10.1371/journal 9(6): e1001242.  \nStuckler, D., McKee, M., Ebrahim, S. & Basu, S.  2012. Manufacturing epidemics: the', metadata={'source': '/content/HLPE-Report-12_AR.pdf', 'page': 215})

In [12]:
model_name_or_path = "nvidia/Llama3-ChatQA-1.5-8B"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

"""model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    #revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device="cuda:0",
    quantize_config=None,
)"""
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.float16, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/51.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/6.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [13]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, and honest assistant. Always answer as helpfully
as possible while being fun. Your answers should not include any harmful,
unethical, dangerous, or illegal content. Please ensure that your
responses are socially unbiased and positive in nature.

Answer with arabic language.

If a question does not make any sense, or is not factually coherent, explain
why instead of answering something not correct. If you don't know the answer to a
question, please don't share false information.
""".strip()

In [14]:
def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

In [15]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.1,
    top_p=0.95, #for faiss to give the best similar 5% tokens
    repetition_penalty=1.15,
    streamer=streamer,
    do_sample=True
)

In [16]:
llm=HuggingFacePipeline(pipeline=text_pipeline,model_kwargs={"temperature":0.1,"language":"ar"})

In [17]:
llm("ما معنى التعلم?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 تعريف التعلم
تعريف التعلم: التعلم هو عملية تغيير في السلوك أو الأفكار أو beiden من خلال التجربة. التعلم يمكن أن يكون عمدًا أو غير عمدٍ، ويمكن أن يحدث على مدار الحياة.
التعلم هو العملية التي تتم بها التغييرات في السلوكيات والافكار والخبرات التي يتم اكتسابها من خلال التجارب والتفاعلات مع البيئة المحيطة. التعلم يمكن أن يكون عمداً أو غير عمداً، ويحدث دائماً في جميع مراحل العمر.

من خلال التعلم، نستطيع تحسين ourselves وتحقيق الأهداف الخاصة بنا. التعلم مهم لنجاحنا الشخصي وللتنمية الاقتصادية والاجتماعية للصالح العام.

في التعليم، التعلم هو الهدف الرئيسي. من خلال التعليم، نستطيع نقل المعرفة والفهم إلى الآخرين وتعزيز التنمية المستدامة في المجتمع.

في الختام، التعلم هو proceso الذي يسمح لنا بالتحسن والتطور كشخصيات ومجتمعات.


'ما معنى التعلم? تعريف التعلم\nتعريف التعلم: التعلم هو عملية تغيير في السلوك أو الأفكار أو beiden من خلال التجربة. التعلم يمكن أن يكون عمدًا أو غير عمدٍ، ويمكن أن يحدث على مدار الحياة.\nالتعلم هو العملية التي تتم بها التغييرات في السلوكيات والافكار والخبرات التي يتم اكتسابها من خلال التجارب والتفاعلات مع البيئة المحيطة. التعلم يمكن أن يكون عمداً أو غير عمداً، ويحدث دائماً في جميع مراحل العمر.\n\nمن خلال التعلم، نستطيع تحسين ourselves وتحقيق الأهداف الخاصة بنا. التعلم مهم لنجاحنا الشخصي وللتنمية الاقتصادية والاجتماعية للصالح العام.\n\nفي التعليم، التعلم هو الهدف الرئيسي. من خلال التعليم، نستطيع نقل المعرفة والفهم إلى الآخرين وتعزيز التنمية المستدامة في المجتمع.\n\nفي الختام، التعلم هو proceso الذي يسمح لنا بالتحسن والتطور كشخصيات ومجتمعات.'

In [18]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. And answer just with arabic language."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

prompt = PromptTemplate(template=template, input_variables=["context","question"])

In [19]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=False,
    chain_type_kwargs={"prompt":prompt},
)

In [31]:
qa_chain("انواع التغذية")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'query': 'انواع التغذية',
 'result': "[INST] <<SYS>>\nUse the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. And answer just with arabic language.\n<</SYS>>\n\n\nالحديثة\n\nوبالتالي ارتفاع العمر المتوقع\n\nQuestion: انواع التغذية\n [/INST]"}

In [21]:
def answer_question(question):
    result_text = llm(question)
    return result_text


In [22]:
import locale
locale.getpreferredencoding = lambda:"UTF-8"

In [23]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00


In [24]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 65.9 MB/s

In [25]:
import gradio as gr

In [26]:
# Create a Gradio interface
iface = gr.Interface(
    fn=answer_question,
    inputs="text", outputs="text",
    title="LLM Question Answerer",
    description="Type a question and get an answer based on the document content."
)

# Run the interface
iface.launch(share=True,debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3f4133b3a2a95ac6a2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3f4133b3a2a95ac6a2.gradio.live


In [27]:
print("\n\n\nالتعلم بمنحنى بياني، سواء كان هذا المنحنى للتعلم الفردي، أو الجمعي للفصل؛ كي نستطيع \nأن نرى مستوى التحصيل، ومستوى الأداء للفصل وللأفراد. \nحدود التعلم \nهل للتعلم حدود؟ نعم، قد نسميها: معوقات، قد نسميها: ميسرات للتعلم، يمكن تقسيم هذه \nالحدود إلى ثلاثة، هي: \n1- الحد المادي، أو البيئي: وهذا دور البيئة والظروف التي تحيط بالكائن الحي، والإعاقات \nالتي تصدر من البيئة، والميسرات التي تضعها البيئة لعملية التعلم؛ حيث تقف البيئة أو\n\nالتعلم الحركي، منها: تعلم السباحة، وركوب الخيل، والجري، والقفز، والكتابة على الآلة  \nالكاتبة، والألعاب الرياضية، والتدريب على الحاسوب الآلي، والصلة بين المهارة والعادة صلة \nوثيقة؛ فركوب الدراجة يمكن أن تكتسب فيه عادة ومهارة معًا، ولكن عادة الضحك بصوت \nعالي في المجتمعات العامة -مثلاً - ليست مهارة؛ فالمهارة معناها: العادات الحركية التي تخدم \nأغراضًا اجتماعية مرغوبًا فيها، كمهارة الكتابة على الحاسوب الآلي، أو لعب التنس.")




التعلم بمنحنى بياني، سواء كان هذا المنحنى للتعلم الفردي، أو الجمعي للفصل؛ كي نستطيع 
أن نرى مستوى التحصيل، ومستوى الأداء للفصل وللأفراد. 
حدود التعلم 
هل للتعلم حدود؟ نعم، قد نسميها: معوقات، قد نسميها: ميسرات للتعلم، يمكن تقسيم هذه 
الحدود إلى ثلاثة، هي: 
1- الحد المادي، أو البيئي: وهذا دور البيئة والظروف التي تحيط بالكائن الحي، والإعاقات 
التي تصدر من البيئة، والميسرات التي تضعها البيئة لعملية التعلم؛ حيث تقف البيئة أو

التعلم الحركي، منها: تعلم السباحة، وركوب الخيل، والجري، والقفز، والكتابة على الآلة  
الكاتبة، والألعاب الرياضية، والتدريب على الحاسوب الآلي، والصلة بين المهارة والعادة صلة 
وثيقة؛ فركوب الدراجة يمكن أن تكتسب فيه عادة ومهارة معًا، ولكن عادة الضحك بصوت 
عالي في المجتمعات العامة -مثلاً - ليست مهارة؛ فالمهارة معناها: العادات الحركية التي تخدم 
أغراضًا اجتماعية مرغوبًا فيها، كمهارة الكتابة على الحاسوب الآلي، أو لعب التنس.
